In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


In [4]:
sns.set_style('whitegrid')
np.random.seed(42)
# pd.set_option('display.max_rows',None)

# For any time T (starting from Jan 1st  2020) in the test set, we have the top 10 CC in time T. For each CC, we train the model using data from 2017 to 2019. And use the model to predict the return of time (T+1) given the data of T-90 to T.

## Detailed Steps:
### Find all the top10 CC from Jan 2020 and train each model using data from 2017 to 2019.
### Then for each day T after Jan 2020, we find the top10 CC and combine the exising models with their data of T-90 to T to predict T+1. 
### Then fit the return of T+1 to into mean-variance model to get the optimal CC allocation
### Then test the performance of the strategy

### Find all the top10 CC from Jan 2020

In [5]:
import ast
def find_all_top10CC(date,top10CC,category):
    df_temp = top10CC.loc[date:][category]
    ls_temp = []
    for i in df_temp:
        ls_temp += ast.literal_eval(i)
    return list(set(ls_temp))

In [6]:
TEST_START = '2020-01-01'
top10CC = pd.read_csv('top10CC_daily.csv').set_index('date')
top10CC_Liquid_test = find_all_top10CC(TEST_START, top10CC, 'top10_Liquid')
top10CC_MarketCap_test = find_all_top10CC(TEST_START, top10CC, 'top10_Market')

### Train each model using data from 2017 to 2019.

In [7]:
# Generating recurrent sequences from time series
def create_univariate_rnn_data(data, window_size):
    n = len(data)
    y = data[window_size:]
    data = data.values.reshape(-1, 1) # make 2D
    X = np.hstack(tuple([data[i: n-j, :] for i, j in enumerate(range(window_size, 0, -1))]))
    return pd.DataFrame(X, index=y.index), y

In [8]:
results_path = Path('results', 'univariate_LSTM')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [9]:
def train_model(CC_name):
    print("-"*150+"\n"+"-"*150+"\n"+" "*75+CC_name+"\n"+"-"*150+"\n"+"-"*150)
    
    # Preprocessing data
    df=pd.read_csv('CC_daily_data.csv').dropna()
    
    dfclose = df[df['ticker']==(CC_name+'-USD')].set_index(['date'])['close'].to_frame()
    dfclose = dfclose[~dfclose.index.duplicated(keep = 'first')]
    dfreturn = (np.log(dfclose/dfclose.shift())
                .dropna()
                .rename(columns = {'close':'return'}))

    # Scaling data
    scaler = MinMaxScaler()
    dfreturn_scaled = pd.Series(scaler.fit_transform(dfreturn).squeeze(),index =dfreturn.index)
    dfreturn_scaled.describe()
    
    window_size = 90
    X,y = create_univariate_rnn_data(dfreturn_scaled, window_size)
    
    # Train-Test split
    X_train = X[:'2020'].values.reshape(-1, window_size, 1)
    y_train = y[:'2020']

    # keep the last year for testing
    X_test = X['2020':].values.reshape(-1, window_size, 1)
    y_test = y['2020':]
    
    # define model architechture
    n_obs, window_size, n_features = X_train.shape
    rnn_return = Sequential([
        LSTM(units = 10, input_shape = (window_size,n_features), name='LSTM'),
        Dense(1, name='Output')
    ])
    
    # define model optimizer
    optimizer = keras.optimizers.RMSprop(lr=.001, rho = .9, epsilon = 1e-08, decay = .0)
    rnn_return.compile(loss = 'mean_squared_error', optimizer = optimizer)
    
    # define checkpointer and early_stoppint point
    rnn_path = (results_path / 'rnn.h5').as_posix()
    checkpointer = ModelCheckpoint(filepath=rnn_path,verbose=1,monitor='val_loss',save_best_only=True)
    early_stopping = EarlyStopping(monitor = 'val_loss',patience = 20, restore_best_weights = True)
    
    # train the model
    lstm_training_return = rnn_return.fit(X_train,y_train, epochs = 150, batch_size = 20, shuffle = True, validation_data=(X_test,y_test),callbacks = [early_stopping,checkpointer],verbose =1)
    
    # prediction
    train_predict_scaled = rnn_return.predict(X_train)
    test_predict_scaled = rnn_return.predict(X_test)
    
    # rescale predictions
    train_predict = pd.Series(scaler.inverse_transform(train_predict_scaled).squeeze(), index=y_train.index)
    test_predict = (pd.Series(scaler.inverse_transform(test_predict_scaled).squeeze(), index=y_test.index))
    
    
    dfreturn['Train Predictions'] = train_predict
    dfreturn['Test Predictions'] = test_predict
    dfreturn = dfreturn.join(train_predict.to_frame('predictions').assign(data='Train')
                            .append(test_predict.to_frame('predictions').assign(data='Test')))
    
    
    model_test = dfreturn.copy().fillna(0)
    tmp = model_test['return']*model_test['predictions'][-30:]
    accuracy = len(tmp[tmp>0])/30
    print(f'Test accuracy: {accuracy:.2f}')
    
    return dfreturn

In [10]:
for CC_name in top10CC_MarketCap_test:
    exec(f"data_{CC_name} = train_model(CC_name)")

------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                           TRX
------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/150
38/38 [==============================] - 32s 67ms/step - loss: 0.0579 - val_loss: 0.0019

Epoch 00001: val_loss improved from inf to 0.00194, saving model to results/univariate_LSTM\rnn.h5
Epoch 2/150
38/38 [==============================] - 0s 12ms/step - loss: 0.0045 - val_loss: 0.0026

Epoch 00002: va

51/51 [==============================] - 1s 11ms/step - loss: 0.0020 - val_loss: 0.0019

Epoch 00011: val_loss did not improve from 0.00147
Epoch 12/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0017 - val_loss: 0.0017

Epoch 00012: val_loss did not improve from 0.00147
Epoch 13/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0023 - val_loss: 0.0019

Epoch 00013: val_loss did not improve from 0.00147
Epoch 14/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0023 - val_loss: 0.0017

Epoch 00014: val_loss did not improve from 0.00147
Epoch 15/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0021 - val_loss: 0.0015

Epoch 00015: val_loss did not improve from 0.00147
Epoch 16/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0023 - val_loss: 0.0016

Epoch 00016: val_loss did not improve from 0.00147
Epoch 17/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0017 - val_loss: 0


Epoch 00062: val_loss improved from 0.00142 to 0.00141, saving model to results/univariate_LSTM\rnn.h5
Epoch 63/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0020 - val_loss: 0.0015

Epoch 00063: val_loss did not improve from 0.00141
Epoch 64/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0021 - val_loss: 0.0014

Epoch 00064: val_loss did not improve from 0.00141
Epoch 65/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0017 - val_loss: 0.0015

Epoch 00065: val_loss did not improve from 0.00141
Epoch 66/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0022 - val_loss: 0.0014

Epoch 00066: val_loss did not improve from 0.00141
Epoch 67/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0020 - val_loss: 0.0015

Epoch 00067: val_loss did not improve from 0.00141
Epoch 68/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0022 - val_loss: 0.0021

Epoch 00068: val_loss did not

51/51 [==============================] - 1s 10ms/step - loss: 0.0014 - val_loss: 0.0014

Epoch 00114: val_loss did not improve from 0.00141
Epoch 115/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0018 - val_loss: 0.0014

Epoch 00115: val_loss did not improve from 0.00141
Epoch 116/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0020 - val_loss: 0.0016

Epoch 00116: val_loss did not improve from 0.00141
Epoch 117/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0021 - val_loss: 0.0014

Epoch 00117: val_loss improved from 0.00141 to 0.00141, saving model to results/univariate_LSTM\rnn.h5
Epoch 118/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0019 - val_loss: 0.0015

Epoch 00118: val_loss did not improve from 0.00141
Epoch 119/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0019 - val_loss: 0.0014

Epoch 00119: val_loss did not improve from 0.00141
Epoch 120/150
51/51 [=================


Epoch 00018: val_loss improved from 0.00402 to 0.00401, saving model to results/univariate_LSTM\rnn.h5
Epoch 19/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0047 - val_loss: 0.0040

Epoch 00019: val_loss did not improve from 0.00401
Epoch 20/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0043 - val_loss: 0.0041

Epoch 00020: val_loss did not improve from 0.00401
Epoch 21/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0051 - val_loss: 0.0040

Epoch 00021: val_loss did not improve from 0.00401
Epoch 22/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0051 - val_loss: 0.0042

Epoch 00022: val_loss did not improve from 0.00401
Epoch 23/150
37/37 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0043

Epoch 00023: val_loss did not improve from 0.00401
Epoch 24/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0046 - val_loss: 0.0040

Epoch 00024: val_loss did not

...

42/42 [==============================] - 0s 12ms/step - loss: 0.0023 - val_loss: 0.0014

Epoch 00038: val_loss did not improve from 0.00133
Epoch 39/150
42/42 [==============================] - 0s 12ms/step - loss: 0.0027 - val_loss: 0.0015

Epoch 00039: val_loss did not improve from 0.00133
Epoch 40/150
42/42 [==============================] - 0s 11ms/step - loss: 0.0023 - val_loss: 0.0014

Epoch 00040: val_loss did not improve from 0.00133
Epoch 41/150
42/42 [==============================] - 0s 11ms/step - loss: 0.0021 - val_loss: 0.0028

Epoch 00041: val_loss did not improve from 0.00133
Epoch 42/150
42/42 [==============================] - 0s 11ms/step - loss: 0.0024 - val_loss: 0.0013

Epoch 00042: val_loss improved from 0.00133 to 0.00132, saving model to results/univariate_LSTM\rnn.h5
Epoch 43/150
42/42 [==============================] - 0s 11ms/step - loss: 0.0024 - val_loss: 0.0016

Epoch 00043: val_loss did not improve from 0.00132
Epoch 44/150
42/42 [=======================

51/51 [==============================] - 1s 11ms/step - loss: 0.0052 - val_loss: 0.0042

Epoch 00006: val_loss did not improve from 0.00279
Epoch 7/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0050 - val_loss: 0.0046

Epoch 00007: val_loss did not improve from 0.00279
Epoch 8/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0050 - val_loss: 0.0028

Epoch 00008: val_loss improved from 0.00279 to 0.00277, saving model to results/univariate_LSTM\rnn.h5
Epoch 9/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0050 - val_loss: 0.0042

Epoch 00009: val_loss did not improve from 0.00277
Epoch 10/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0051 - val_loss: 0.0030

Epoch 00010: val_loss did not improve from 0.00277
Epoch 11/150
51/51 [==============================] - 1s 10ms/step - loss: 0.0046 - val_loss: 0.0049

Epoch 00011: val_loss did not improve from 0.00277
Epoch 12/150
51/51 [==========================

37/37 [==============================] - 0s 12ms/step - loss: 0.0022 - val_loss: 0.0020

Epoch 00035: val_loss did not improve from 0.00195
Epoch 36/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0020

Epoch 00036: val_loss did not improve from 0.00195
Epoch 37/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 0.0023

Epoch 00037: val_loss did not improve from 0.00195
Epoch 38/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0023 - val_loss: 0.0025

Epoch 00038: val_loss did not improve from 0.00195
Epoch 39/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0022 - val_loss: 0.0021

Epoch 00039: val_loss did not improve from 0.00195
Epoch 40/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 0.0026

Epoch 00040: val_loss did not improve from 0.00195
Epoch 41/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 0

37/37 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0019

Epoch 00137: val_loss did not improve from 0.00191
Epoch 138/150
37/37 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0025

Epoch 00138: val_loss did not improve from 0.00191
Epoch 139/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0019 - val_loss: 0.0020

Epoch 00139: val_loss did not improve from 0.00191
Epoch 140/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0019 - val_loss: 0.0019

Epoch 00140: val_loss did not improve from 0.00191
Epoch 141/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0017 - val_loss: 0.0020

Epoch 00141: val_loss did not improve from 0.00191
Epoch 142/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0020

Epoch 00142: val_loss did not improve from 0.00191
Epoch 143/150
37/37 [==============================] - 0s 11ms/step - loss: 0.0019 - val_l


Epoch 00033: val_loss did not improve from 0.00343
Epoch 34/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0046 - val_loss: 0.0039

Epoch 00034: val_loss did not improve from 0.00343
Epoch 35/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0041 - val_loss: 0.0034

Epoch 00035: val_loss did not improve from 0.00343
Epoch 36/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0039 - val_loss: 0.0036

Epoch 00036: val_loss did not improve from 0.00343
Epoch 37/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0035 - val_loss: 0.0038

Epoch 00037: val_loss did not improve from 0.00343
Epoch 38/150
51/51 [==============================] - 1s 10ms/step - loss: 0.0041 - val_loss: 0.0035

Epoch 00038: val_loss did not improve from 0.00343
Epoch 39/150
51/51 [==============================] - 1s 11ms/step - loss: 0.0039 - val_loss: 0.0035

Epoch 00039: val_loss did not improve from 0.00343
Epoch 40/150
51/51 [==========

### Fit the return of T+1 to into mean-variance model to get the optimal CC allocation

In [55]:
def find_top10CC_historical_returns(CC_name,CC_daily_data):
    dfclose = CC_daily_data[CC_daily_data['ticker']==(CC_name+'-USD')].set_index(['date'])['close'].to_frame()
    dfclose = dfclose[~dfclose.index.duplicated(keep = 'first')]
    dfreturn = (np.log(dfclose/dfclose.shift())
                .dropna()
                .rename(columns = {'close':CC_name}))
    return dfreturn

In [87]:
from pypfopt.efficient_frontier import EfficientFrontier
def find_optimal_weights(Returns,CovarianceMatrix):
    ef = EfficientFrontier(Returns, CovarianceMatrix, weight_bounds=(0,1))
    weights = ef.max_sharpe(risk_free_rate= 0.0)
    weights = ef.clean_weights()
    return weights

In [179]:
TEST_START = '2020-01-01'
TEMP_DATE = '2020-01-02'
CC_daily_data=pd.read_csv('CC_daily_data.csv').dropna()
capital = 1
for date in top10CC.loc[TEST_START:].index:
    if date not in data_BTC.index:
        continue
    
    # Initialize variables
    top10CC_predicted_return = pd.DataFrame(columns = ['Ticker','Predicted_Return'])
    top10CC_historical_return = pd.DataFrame()
    
    # Get expected(predicted) returns on the "date", and the previous real returns before the date
    for  CC_name in ast.literal_eval(top10CC.loc[date]['top10_Market']):
        exec(f"return_temp = data_{CC_name}.loc[date]['predictions']")
        top10CC_predicted_return = top10CC_predicted_return.append({'Ticker':CC_name,'Predicted_Return':return_temp},ignore_index = True)
        top10CC_historical_return = pd.concat([top10CC_historical_return,find_top10CC_historical_returns(CC_name,CC_daily_data)],axis = 1).dropna()
        
    top10CC_predicted_return = top10CC_predicted_return.set_index('Ticker')
    top10CC_real_return = top10CC_historical_return.loc[date].to_frame()
    top10CC_historical_return = top10CC_historical_return.loc[:date][:-1]
    if not top10CC_predicted_return[top10CC_predicted_return>0].any()[0]:
        continue
    # Get the optimal weight of top10CC on the "date"
    weights = find_optimal_weights(top10CC_predicted_return['Predicted_Return'],top10CC_historical_return.cov())
    
    # Calcualte the real gain/loss on the "date"
    return_sum = (np.array(list(weights.values())).reshape(-1,1) * top10CC_real_return).sum()[0]
    capital *= (1+return_sum)
    
    print(f"Date: {date}   Return: {return_sum:4f}  Capital: {capital:4f}")

Date: 2020-01-01   Return: 0.012019  Capital: 1.012019
Date: 2020-01-02   Return: -0.024367  Capital: 0.987360
Date: 2020-01-03   Return: 0.050172  Capital: 1.036897
Date: 2020-01-04   Return: 0.013603  Capital: 1.051002
Date: 2020-01-05   Return: 0.025046  Capital: 1.077326
Date: 2020-01-06   Return: 0.047568  Capital: 1.128572
Date: 2020-01-07   Return: -0.005847  Capital: 1.121973
Date: 2020-01-08   Return: -0.015320  Capital: 1.104785
Date: 2020-01-09   Return: -0.011349  Capital: 1.092247
Date: 2020-01-10   Return: 0.021532  Capital: 1.115765
Date: 2020-01-11   Return: -0.015594  Capital: 1.098366
Date: 2020-01-12   Return: 0.023694  Capital: 1.124390
Date: 2020-01-13   Return: -0.020264  Capital: 1.101606
Date: 2020-01-14   Return: 0.119607  Capital: 1.233366
Date: 2020-01-15   Return: -0.022798  Capital: 1.205247
Date: 2020-01-16   Return: -0.009910  Capital: 1.193304
Date: 2020-01-17   Return: 0.054128  Capital: 1.257895
Date: 2020-01-18   Return: -0.022029  Capital: 1.230186
D

Date: 2020-06-05   Return: 0.006812  Capital: 1.723294
Date: 2020-06-06   Return: 0.007274  Capital: 1.735828
Date: 2020-06-07   Return: -0.002588  Capital: 1.731335
Date: 2020-06-08   Return: -0.035603  Capital: 1.669695
Date: 2020-06-09   Return: 0.001486  Capital: 1.672176
Date: 2020-06-10   Return: -0.102941  Capital: 1.500040
Date: 2020-06-11   Return: 0.054159  Capital: 1.581281
Date: 2020-06-12   Return: -0.006634  Capital: 1.570791
Date: 2020-06-13   Return: -0.029275  Capital: 1.524806
Date: 2020-06-14   Return: 0.021131  Capital: 1.557026
Date: 2020-06-15   Return: 0.008713  Capital: 1.570593
Date: 2020-06-16   Return: 0.053816  Capital: 1.655117
Date: 2020-06-17   Return: -0.015349  Capital: 1.629713
Date: 2020-06-18   Return: -0.031787  Capital: 1.577909
Date: 2020-06-19   Return: -0.003322  Capital: 1.572667
Date: 2020-06-20   Return: -0.010950  Capital: 1.555447
Date: 2020-06-21   Return: 0.063383  Capital: 1.654035
Date: 2020-06-22   Return: -0.007925  Capital: 1.640927


Date: 2020-11-04   Return: 0.015810  Capital: 2.933418
Date: 2020-11-05   Return: 0.050107  Capital: 3.080404
Date: 2020-11-06   Return: 0.056979  Capital: 3.255921
Date: 2020-11-07   Return: -0.060444  Capital: 3.059121
Date: 2020-11-08   Return: 0.065497  Capital: 3.259483
Date: 2020-11-09   Return: -0.014226  Capital: 3.213113
Date: 2020-11-10   Return: 0.002038  Capital: 3.219662
Date: 2020-11-11   Return: 0.010701  Capital: 3.254117
Date: 2020-11-12   Return: 0.000232  Capital: 3.254872
Date: 2020-11-13   Return: 0.012784  Capital: 3.296483
Date: 2020-11-14   Return: -0.019139  Capital: 3.233391
Date: 2020-11-15   Return: -0.023565  Capital: 3.157196
Date: 2020-11-16   Return: 0.042935  Capital: 3.292749
Date: 2020-11-17   Return: 0.058426  Capital: 3.485131
Date: 2020-11-18   Return: -0.029217  Capital: 3.383305
Date: 2020-11-19   Return: 0.042121  Capital: 3.525814
Date: 2020-11-20   Return: 0.029802  Capital: 3.630892
Date: 2020-11-21   Return: 0.068039  Capital: 3.877933
Date:

In [189]:
import datetime
CC100_Index = pd.read_csv("indexReturns-BITW100.csv")
CC100_Index['date'] = pd.to_datetime(CC100_Index['date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('date').set_index('date')
CC100_Index = CC100_Index.truncate(before = datetime.datetime(2020,1,1),after = datetime.datetime(2020,12,31))

In [188]:
return_benchmark = (CC100_Index.iloc[-1]/CC100_Index.iloc[0])[0] -1
return_benchmark

2.726031544384666